### 2体問題の軌道サンプルを作る関数
- 2体問題の軌道伝播をM本
- それをランダムなtiからtjまでの時間に区切る
- 初期位置・速度，終端位置・速度を出力する

In [ ]:
# EOMの定義
function twobody_with_sun!(dxdt, x, p, t)
   # x = [x, y, z, vx, vy, vz]
   # dx/dt
   r = norm(x[1:3]) # x[1:3] = [x, y, z], つまりrは位置ベクトルのノルム
   dxdt[1:3] = x[4:6] # 速度ベクトル
   dxdt[4:6] = -ssd.GM["SUN"] * x[1:3] / (r^3) # 万有引力の法則
end

# 2体問題による軌道伝播
function propagation_twoBody(twobody_problem, r_sat_start, v_sat_start, T_start, T_end)
   # 初期状態の定義
   x0 = [r_sat_start; v_sat_start]

   tspan = (T_start, T_end)

   prob = ODEProblem(twobody_problem, x0, tspan)
   sol = solve(prob, Tsit5(), reltol=1e-8, abstol=1e-8)
   r_sat_end = sol.u[end][1:3]
   v_sat_end = sol.u[end][4:6]
   return r_sat_end, v_sat_end
end

# 2体問題による軌道伝播をM種類作成し、それぞれの軌道をランダムな時間t_iからt_jの時間で区切り、その初期位置・速度、終端位置・速度を出力
using Random

function generate_orbit_samples(M, twobody_problem, r_sat_start, v_sat_start, T_start, T_end)
   # 初期位置・速度、終端位置・速度を保存する配列
   initial_positions = []
   initial_velocities = []
   final_positions = []
   final_velocities = []

   for i in 1:M
      # ランダムなtiからtjまでの時間を生成
      t_i = rand(T_start:T_end)
      t_j = rand(t_i:T_end)

      # 軌道伝播を実行
      r_sat_end, v_sat_end = propagation_twoBody(twobody_problem, r_sat_start, v_sat_start, t_i, t_j)

      # 初期位置・速度、終端位置・速度を保存
      push!(initial_positions, r_sat_start)
      push!(initial_velocities, v_sat_start)
      push!(final_positions, r_sat_end)
      push!(final_velocities, v_sat_end)
   end

   return initial_positions, initial_velocities, final_positions, final_velocities
end
